In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/raw_data/cosqa_queries_code_corpus.csv')

In [3]:
df.shape

(20604, 4)

In [ ]:
from vllm import LLM
from vllm.sampling_params import SamplingParams
import pandas as pd

df1 = df.head(10)

# 2. Initialize vLLM client once
client = LLM(model="/datasets/ai/llama3/hub/models--meta-llama--Llama-3.2-1B-Instruct/snapshots/9213176726f574b556790deb65791e0c5aa438b6", dtype="half")

# 3. Define deterministic sampling
params = SamplingParams(
    max_tokens=2,
    temperature=0.0,
    top_p=1.0
    )

# 4. Row-wise function that builds the prompt, calls vLLM, and returns “yes”/“no”
def impl_check(row):
    prompt = f"""
    Docstring:
    {row['doc']}

    Code:
    {row['code']}

    Question: Does the code implement the behavior described in the docstring? Answer “yes” or “no” only.
    Answer:\n
    """

    response = client.generate([{"prompt": prompt}], sampling_params=params)
    # grab the single-token reply, strip whitespace, force lowercase
    return response[0].outputs[0].text.strip().lower()

/home/anamikaghosh_umass_edu/.conda/envs/gpu-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-20 18:04:30,455	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 04-20 18:04:32 __init__.py:207] Automatically detected platform cuda.
WARNING 04-20 18:04:32 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 04-20 18:04:46 config.py:549] This model supports multiple tasks: {'embed', 'score', 'generate', 'reward', 'classify'}. Defaulting to 'generate'.
WARNING 04-20 18:04:46 arg_utils.py:1187] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 04-20 18:04:46 config.py:1555] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 04-20 18:04:46 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='//datasets/ai/llama3/hub/models--meta-llama--Llama-3.2-1B-Instruct/snapshots/9213176726f574b556790deb65791e0c5aa438b6', speculative_config=None, tokenizer='//datasets/ai/llama3/hub/models--meta-llama--Llama-3

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:03<00:00,  3.05s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:03<00:00,  3.05s/it]



INFO 04-20 18:04:53 model_runner.py:1115] Loading model weights took 2.3185 GB
INFO 04-20 18:04:54 worker.py:267] Memory profiling takes 0.73 seconds
INFO 04-20 18:04:54 worker.py:267] the current vLLM instance can use total_gpu_memory (47.27GiB) x gpu_memory_utilization (0.90) = 42.54GiB
INFO 04-20 18:04:54 worker.py:267] model weights take 2.32GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.18GiB; the rest of the memory reserved for KV Cache is 38.99GiB.
INFO 04-20 18:04:54 executor_base.py:111] # cuda blocks: 79845, # CPU blocks: 8192
INFO 04-20 18:04:54 executor_base.py:116] Maximum concurrency for 131072 tokens per request: 9.75x


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.44 GiB. GPU 0 has a total capacity of 47.27 GiB of which 433.19 MiB is free. Process 94173 has 41.83 GiB memory in use. Including non-PyTorch memory, this process has 5.01 GiB memory in use. Of the allocated memory 4.76 GiB is allocated by PyTorch, and 29.71 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# 5. Apply it across your DataFrame
df['validation_flag'] = df.apply(impl_check, axis=1)

# 6. Now df has a new column 'implements' with “yes” or “no”
print(df.head())

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	

  query_id                                              doc corpus_id  \
0       q1                python code to write bool value 1        d1   
1       q2             "python how to manipulate clipboard"        d2   
2       q3                    python colored output to html        d3   
3       q4  python "create directory" using "relative path"        d4   
4       q5                        python column of an array        d5   

                                                code validation_flag  
0  def writeBoolean(self, n):\n        """\n     ...             yes  
1  def paste(xsel=False):\n    """Returns system ...             yes  
2  def _format_json(data, theme):\n    """Pretty ...              no  
3  def create_path(path):\n    """Creates a absol...             yes  
4  def _vector_or_scalar(x, type='row'):\n    """...              no  



/tmp/ipykernel_94173/462374724.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['validation_flag'] = df1.apply(impl_check, axis=1)


In [11]:
pd.set_option('display.max_colwidth', 500)
df1[df1['validation_flag']=='no']

,query_id,doc,corpus_id,code,validation_flag
2,q3,python colored output to html,d3,"def _format_json(data, theme):\n """"""Pretty print a dict as a JSON, with colors if pygments is present.""""""\n output = json.dumps(data, indent=2, sort_keys=True)\n\n if pygments and sys.stdout.isatty():\n style = get_style_by_name(theme)\n formatter = Terminal256Formatter(style=style)\n return pygments.highlight(output, JsonLexer(), formatter)\n\n return output",no
4,q5,python column of an array,d5,"def _vector_or_scalar(x, type='row'):\n """"""Convert an object to either a scalar or a row or column vector.""""""\n if isinstance(x, (list, tuple)):\n x = np.array(x)\n if isinstance(x, np.ndarray):\n assert x.ndim == 1\n if type == 'column':\n x = x[:, None]\n return x",no
6,q7,python combine wav file into one as separate channels,d7,"def data_from_file(file):\n """"""Return (first channel data, sample frequency, sample width) from a .wav\n file.""""""\n fp = wave.open(file, 'r')\n data = fp.readframes(fp.getnframes())\n channels = fp.getnchannels()\n freq = fp.getframerate()\n bits = fp.getsampwidth()\n\n # Unpack bytes -- warning currently only tested with 16 bit wavefiles. 32\n # bit not supported.\n data = struct.unpack(('%sh' % fp.getnframes()) * channels, data)\n\n # Only use first channel...",no
7,q8,+how to use range with a dictionary python,d8,"def source_range(start, end, nr_var_dict):\n """"""\n Given a range of source numbers, as well as a dictionary\n containing the numbers of each source, returns a dictionary\n containing tuples of the start and end index\n for each source variable type.\n """"""\n\n return OrderedDict((k, e-s)\n for k, (s, e)\n in source_range_tuple(start, end, nr_var_dict).iteritems())",no
8,q9,python compare timespan to number,d9,"def timespan(start_time):\n """"""Return time in milliseconds from start_time""""""\n\n timespan = datetime.datetime.now() - start_time\n timespan_ms = timespan.total_seconds() * 1000\n return timespan_ms",no
9,q10,1d array in char datatype in python,d10,"def _convert_to_array(array_like, dtype):\n """"""\n Convert Matrix attributes which are array-like or buffer to array.\n """"""\n if isinstance(array_like, bytes):\n return np.frombuffer(array_like, dtype=dtype)\n return np.asarray(array_like, dtype=dtype)",no


In [ ]:
# New row data as a dictionary
new_row = {'query_id': 'test', 'doc': 'python add two numbers', 'corpus_id': 'test', 'code': "def add_numbers(a, b):\n  return a + b
"}

# Add the new row using loc
df1.loc[len(df1)] = new_row

In [14]:
print(
'''''''''def _vector_or_scalar(x, type='row'):\n """Convert an object to either a scalar or a row or column vector."""\n if isinstance(x, (list, tuple)):\n x = np.array(x)\n if isinstance(x, np.ndarray):\n assert x.ndim == 1\n if type == 'column':\n x = x[:, None]\n return x'''''''''
)

def _vector_or_scalar(x, type='row'):
 """Convert an object to either a scalar or a row or column vector."""
 if isinstance(x, (list, tuple)):
 x = np.array(x)
 if isinstance(x, np.ndarray):
 assert x.ndim == 1
 if type == 'column':
 x = x[:, None]
 return x


In [5]:
df['validation_flag']

KeyError: 'validation_flag'

In [1]:
import pandas as pd
valid_df = pd.read_csv('/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/pre_processing_COSQA/COSQA_validation_query_code.csv')

In [3]:
valid_df['validation_flag'].value_counts()

validation_flag
no                13291
yes                6236
the code            191
"yes                131
```python           127
                  ...  
python strings        1
python syntax         1
python tkinter        1
python trim           1
python sets           1
Name: count, Length: 241, dtype: int64

In [4]:
label_df = pd.read_json("/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CodeXGLUE/Text-Code/NL-code-search-WebQuery/CoSQA/cosqa-all.json")

In [5]:
label_df.head()

,idx,doc,code,code_tokens,docstring_tokens,label
0,cosqa-train-18103,python remove all empty items in list,def remove_empty_text(utterances: List[Utteran...,def remove_empty_text ( utterances : List [ Ut...,Remove empty utterances from a list of utteran...,0
1,cosqa-train-11841,python return property objectno not value,"def value(self):\n """"""Value of property...",def value ( self ) : if self . _prop . fget is...,Value of property .,1
2,cosqa-train-3015,python not null dict,"def purge_dict(idict):\n """"""Remove null ite...",def purge_dict ( idict ) : odict = { } for key...,Remove null items from a dictionary,0
3,cosqa-train-5561,how to turn of traceback in python,"def format_exception(e):\n """"""Returns a str...",def format_exception ( e ) : from . utils . pr...,Returns a string containing the type and text ...,0
4,cosqa-train-9028,generate list of fixed size python,"def batch(items, size):\n """"""Batches a list...","def batch ( items , size ) : return [ items [ ...",Batches a list into a list of lists with sub -...,1


In [11]:
label_df['doc'].nunique()

20604

In [15]:
temp = pd.merge(valid_df, label_df[['doc','label']], on='doc')

In [22]:
pd.DataFrame(temp.groupby(['validation_flag','label'])['query_id'].count()).to_csv('/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/pre_processing_COSQA/COSQA_validation_analysis.csv')